<a href="https://colab.research.google.com/github/rahiakela/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/blob/10-introduction-to-artificial-neural-networks-with-keras/visualization_using_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using TensorBoard for Visualization

TensorBoard is a great interactive visualization tool that you can use to view the learning curves during training, compare learning curves between multiple runs, visualize the computation graph, analyze training statistics, view images generated by your model, visualize complex multidimensional data projected down to 3D and automatically clustered for you, and more! This tool is installed automatically when you install TensorFlow, so you already have it.

To use it, you must modify your program so that it outputs the data you want to visualize to special binary log files called event files . Each binary data record is called a summary . The TensorBoard server will monitor the log directory, and it will automatically pick up the changes and update the visualizations: this allows you to visualize live data (with a short delay), such as the learning curves during training.

## Setup

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


## Load database

Let’s switch to the California housing problem and tackle it using a regression neural network. For simplicity, we will use Scikit-Learn’s fetch_california_housing() function to load the data.

After loading the data, we split it into a training set, a validation set, and a test set, and we scale all the features:

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# load dataset
housing = fetch_california_housing()

# split dataset into traing and test set 
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# prepare validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# scale all the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

Let’s start by defining the root log directory we will use for our TensorBoard logs, plus a small function that will generate a subdirectory path based on the current date and time so that it’s different at every run.

In [0]:
import os

root_logdir = os.path.join(os.curdir, 'my_logs')

In [6]:
def get_run_logdir():
  import time
  run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
  return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir 

'./my_logs/run_2019_11_26-09_19_14'

## Build the model with TensorBoard callback

In [0]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

## Running TensorBoard

To start the TensorBoard server, one option is to open a terminal, if needed activate the virtualenv where you installed TensorBoard, go to this notebook's directory, then type:

$ tensorboard --logdir=./my_logs --port=6006
You can then open your web browser to localhost:6006 and use TensorBoard. Once you are done, press Ctrl-C in the terminal window, this will shutdown the TensorBoard server.

Alternatively, you can use TensorBoard directly within Jupyter, by running the following commands. The first line loads the TensorBoard extension, and the second line starts a TensorBoard server on port 6006 (unless it is already started) and connects to it:

reference: 
https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab

In [0]:
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [21]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2019-11-26 10:11:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.235.200.97, 52.3.53.115, 3.221.56.183, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.235.200.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.1MB/s    in 0.4s    

2019-11-26 10:11:48 (37.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [23]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://4f89514d.ngrok.io


In [28]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

Train on 11610 samples, validate on 3870 samples
Epoch 1/30
11610/11610 [==============================] - 1s 58us/sample - loss: 0.3727 - val_loss: 0.3587
Epoch 2/30
11610/11610 [==============================] - 1s 60us/sample - loss: 0.3713 - val_loss: 0.3479
Epoch 3/30
11610/11610 [==============================] - 1s 54us/sample - loss: 0.3698 - val_loss: 0.3481
Epoch 4/30
11610/11610 [==============================] - 1s 55us/sample - loss: 0.3682 - val_loss: 0.3455
Epoch 5/30
11610/11610 [==============================] - 1s 54us/sample - loss: 0.3669 - val_loss: 0.3483
Epoch 6/30
11610/11610 [==============================] - 1s 55us/sample - loss: 0.3655 - val_loss: 0.3583
Epoch 7/30
11610/11610 [==============================] - 1s 55us/sample - loss: 0.3641 - val_loss: 0.3423
Epoch 8/30
11610/11610 [==============================] - 1s 56us/sample - loss: 0.3633 - val_loss: 0.3738
Epoch 9/30
11610/11610 [==============================] - 1s 56us/sample - loss: 0.3620 - val_l

In [0]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [0]:

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.05))

In [31]:
run_logdir2 = get_run_logdir()
run_logdir2

'./my_logs/run_2019_11_26-10_30_34'

In [32]:

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir2)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Train on 11610 samples, validate on 3870 samples
Epoch 1/30
11610/11610 [==============================] - 1s 86us/sample - loss: 0.7860 - val_loss: 3.9608
Epoch 2/30
11610/11610 [==============================] - 1s 59us/sample - loss: 0.4406 - val_loss: 19.3301
Epoch 3/30
11610/11610 [==============================] - 1s 56us/sample - loss: 0.3807 - val_loss: 1.9304
Epoch 4/30
11610/11610 [==============================] - 1s 55us/sample - loss: 0.4081 - val_loss: 0.3692
Epoch 5/30
11610/11610 [==============================] - 1s 56us/sample - loss: 0.3796 - val_loss: 0.3266
Epoch 6/30
11610/11610 [==============================] - 1s 58us/sample - loss: 0.3664 - val_loss: 0.3184
Epoch 7/30
11610/11610 [==============================] - 1s 57us/sample - loss: 0.3655 - val_loss: 0.3169
Epoch 8/30
11610/11610 [==============================] - 1s 55us/sample - loss: 0.3443 - val_loss: 0.3244
Epoch 9/30
11610/11610 [==============================] - 1s 56us/sample - loss: 0.3387 - val_

Notice how TensorBoard now sees two runs, and you can compare the learning curves.

Check out the other available logging options:

In [33]:
help(keras.callbacks.TensorBoard.__init__)

Help on function __init__ in module tensorflow.python.keras.callbacks:

__init__(self, log_dir='logs', histogram_freq=0, write_graph=True, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=0, embeddings_metadata=None, **kwargs)
    Initialize self.  See help(type(self)) for accurate signature.

